<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastian Avendaño
- Nombre de alumno 2: Felipe Urrutia


### **Link de repositorio de GitHub:** `https://github.com/furrutiav/lab-mds-2022`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [56]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# UMAP
!pip install umap-learn

https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [57]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [132]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


In [133]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17588 entries, 0 to 17587
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               17588 non-null  object
 1   Nationality        17588 non-null  object
 2   National_Position  1075 non-null   object
 3   Club_Position      17587 non-null  object
 4   Height             17588 non-null  int64 
 5   Weight             17588 non-null  int64 
 6   Preffered_Foot     17588 non-null  object
 7   Age                17588 non-null  int64 
 8   Work_Rate          17588 non-null  object
 9   Weak_foot          17588 non-null  int64 
 10  Skill_Moves        17588 non-null  int64 
 11  Ball_Control       17588 non-null  int64 
 12  Dribbling          17588 non-null  int64 
 13  Marking            17588 non-null  int64 
 14  Sliding_Tackle     17588 non-null  int64 
 15  Standing_Tackle    17588 non-null  int64 
 16  Aggression         17588 non-null  int64

## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [134]:
### Código aquí ###
df_players["LABEL"] = df_players["National_Position"].apply(lambda x: 1 if str(x)!= "nan" else 0)
df_players["LABEL"]

0        1
1        1
2        1
3        1
4        1
        ..
17583    0
17584    0
17585    0
17586    0
17587    0
Name: LABEL, Length: 17588, dtype: int64

In [135]:
df_players["LABEL"].value_counts()

0    16513
1     1075
Name: LABEL, dtype: int64

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

### Genere un ColumnTransformer enfocado en preprocesar los datos.

In [136]:
### Código aquí ###
cols = df_players.columns
cols_categorical = ["Nationality", "Club_Position", "Preffered_Foot", "Work_Rate"]
cols_num = [c for c in cols if (c not in cols_categorical) and (c not in ["LABEL", "Name", "National_Position"])]

ct = ColumnTransformer([("Standard", StandardScaler(), cols_num), ("OneHotEncoder", OneHotEncoder(handle_unknown='ignore'), cols_categorical)])

### Indicar y Justificar que preprocesamiento utiliza sobre cada columna. (PENDIENTE)

In [25]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

### Crear 3 pipelines con diferentes clasificadores.
Para seleccionar los clasificadores base sus decisiones en la siguiente guía (PENDIENTE)
No entrenar los pipelines aún.

In [137]:
linear_svc = LinearSVC(class_weight="balanced")
pipe_linear_svc = Pipeline([
    ('scaler', ct),
    ('classifier', linear_svc)
    ])

knn = KNeighborsClassifier(n_neighbors=3)
pipe_knn = Pipeline([
    ('scaler', ct),
    ('classifier', knn)
    ])

rnd_forest = RandomForestClassifier(class_weight="balanced")
pipe_rnd_forest = Pipeline([
    ('scaler', ct),
    ('classifier', rnd_forest)
    ])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

### Separar el conjunto de datos en entrenamiento y de prueba.

In [138]:
### Código aquí ###
X, y = df_players[cols_categorical+cols_num], df_players["LABEL"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Entrenar los pipelines

In [139]:
%%time
pipe_linear_svc.fit(X_train, y_train)
pipe_knn.fit(X_train, y_train)
pipe_rnd_forest.fit(X_train, y_train)

C:\Users\felip\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Wall time: 6.81 s


Pipeline(steps=[('scaler',
                 ColumnTransformer(transformers=[('Standard', StandardScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Weak_foot', 'Skill_Moves',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                              

In [140]:
%%time
linear_svc_pred = pipe_linear_svc.predict(X_test)
knn_pred = pipe_knn.predict(X_test)
rnd_forest_pred = pipe_rnd_forest.predict(X_test)

Wall time: 7.34 s


### Utilizar las métricas para evaluar los modelos generados.

In [141]:
print(classification_report(y_test, linear_svc_pred))

              precision    recall  f1-score   support

           0       0.99      0.89      0.94      5461
           1       0.32      0.87      0.47       344

    accuracy                           0.88      5805
   macro avg       0.66      0.88      0.70      5805
weighted avg       0.95      0.88      0.91      5805



In [142]:
print(classification_report(y_test, knn_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5461
           1       0.40      0.11      0.17       344

    accuracy                           0.94      5805
   macro avg       0.67      0.55      0.57      5805
weighted avg       0.91      0.94      0.92      5805



In [143]:
print(classification_report(y_test, rnd_forest_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5461
           1       0.55      0.03      0.06       344

    accuracy                           0.94      5805
   macro avg       0.75      0.52      0.52      5805
weighted avg       0.92      0.94      0.92      5805



**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

### Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.

In [144]:
### Código aquí ###
dic_replace_label = {
'ST': "ataque", 
'CF': "ataque",
'RW': "central_ataque",
'CAM': "central_ataque",
'LW': "central_ataque", 
'RM': "central",
'CM': "central",
'LM': "central",
'RWB': "central_defensa",
'CDM': "central_defensa",
'LWB': "central_defensa",
'RB': "defensa",
'CB': "defensa",
'LB': "defensa",
'GK': "arquero"
}

In [145]:
sub_df_players = df_players[df_players["Club_Position"].isin(dic_replace_label.keys())]
sub_df_players["LABEL"] = sub_df_players["Club_Position"].apply(lambda x: dic_replace_label[x])

<ipython-input-145-7711334385c3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df_players["LABEL"] = sub_df_players["Club_Position"].apply(lambda x: dic_replace_label[x])


### Cuente cuantos por clase quedan.

In [146]:
sub_df_players["LABEL"].value_counts()

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: LABEL, dtype: int64

### Entrene el nuevo pipeline y ejecute una evaluación de este. 

In [151]:
### Código aquí ###
cols = sub_df_players.columns
cols_categorical = ["National_Position", "Nationality", "Preffered_Foot", "Work_Rate"]
cols_num = [c for c in cols if (c not in cols_categorical) and (c not in ["LABEL", "Club_Position", "Name"])]

ct = ColumnTransformer([("Standard", StandardScaler(), cols_num), ("OneHotEncoder", OneHotEncoder(handle_unknown='ignore'), cols_categorical)])

In [152]:
linear_svc = LinearSVC(class_weight="balanced")
pipe_linear_svc = Pipeline([
    ('scaler', ct),
    ('classifier', linear_svc)
    ])

knn = KNeighborsClassifier(n_neighbors=3)
pipe_knn = Pipeline([
    ('scaler', ct),
    ('classifier', knn)
    ])

rnd_forest = RandomForestClassifier(class_weight="balanced")
pipe_rnd_forest = Pipeline([
    ('scaler', ct),
    ('classifier', rnd_forest)
    ])

In [153]:
### Código aquí ###
X, y = sub_df_players[cols_categorical+cols_num], sub_df_players["LABEL"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [154]:
%%time
pipe_linear_svc.fit(X_train, y_train)
pipe_knn.fit(X_train, y_train)
pipe_rnd_forest.fit(X_train, y_train)

C:\Users\felip\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Wall time: 2.14 s


Pipeline(steps=[('scaler',
                 ColumnTransformer(transformers=[('Standard', StandardScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Weak_foot', 'Skill_Moves',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                              

In [155]:
%%time
linear_svc_pred = pipe_linear_svc.predict(X_test)
knn_pred = pipe_knn.predict(X_test)
rnd_forest_pred = pipe_rnd_forest.predict(X_test)

Wall time: 448 ms


In [156]:
print(classification_report(y_test, linear_svc_pred))

                 precision    recall  f1-score   support

        arquero       1.00      0.99      1.00       200
         ataque       0.78      0.86      0.81       145
        central       0.60      0.45      0.52       316
 central_ataque       0.44      0.44      0.44       183
central_defensa       0.26      0.41      0.32        68
        defensa       0.82      0.87      0.84       388

       accuracy                           0.70      1300
      macro avg       0.65      0.67      0.66      1300
   weighted avg       0.71      0.70      0.70      1300



In [157]:
print(classification_report(y_test, knn_pred))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       200
         ataque       0.62      0.83      0.71       145
        central       0.50      0.52      0.51       316
 central_ataque       0.40      0.26      0.32       183
central_defensa       0.35      0.16      0.22        68
        defensa       0.80      0.89      0.84       388

       accuracy                           0.68      1300
      macro avg       0.61      0.61      0.60      1300
   weighted avg       0.66      0.68      0.66      1300



In [158]:
print(classification_report(y_test, rnd_forest_pred))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       200
         ataque       0.80      0.85      0.82       145
        central       0.61      0.64      0.63       316
 central_ataque       0.55      0.34      0.42       183
central_defensa       0.60      0.13      0.22        68
        defensa       0.78      0.98      0.87       388

       accuracy                           0.75      1300
      macro avg       0.72      0.66      0.66      1300
   weighted avg       0.73      0.75      0.73      1300



### Comente los resultados obtenidos.

**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

### Explique en que consiste la métrica 𝑅2

El coeficiente de deterimanacion, o tambien conocido como $R^2$, es una metrica que mide el desempeño de un modelo que predice una variable continua. Esta metrica, tiene valores entre cero y uno. Donde aquellos valores mas grandes son mejores. Por otro lado, $R^2$ es igual a uno si y solo si la prediccion es igual que a los datos de referencia, i.e. prediccion perfecta. Para un conjunto de datos $\{y_i\}_i$ y respectivos valores predichos $\{\hat{y_i}\}_i$, la metrica esta definida por

$$R^2 = 1 - \frac{\sum_i (y_i - \hat{y}_i)^2}{\sum_i (y_i - \bar{y})^2}$$

donde $\bar{y}$ corresponde al promedio de los datos $\{y_i\}_i$.

### Generar un pipeline para la regresión.

In [159]:
df_salarios = pd.read_csv('salarios.csv')
df_salarios

,Unnamed: 0,Player,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0
...,...,...,...
2529,1527,Simone Ghidotti,283.0
2530,2539,Valentin Gendrey,100.0
2531,1001,John Yeboah,70.0
2532,1000,Antonis Aidonis,70.0


In [160]:
df_players_salarios = df_salarios.copy().rename(columns={"Player": "Name"})
df_players_salarios = df_players_salarios.merge(df_players, on="Name", how="left")
df_players_salarios = df_players_salarios.dropna()
df_players_salarios = df_players_salarios.drop(columns=["Unnamed: 0"])

In [161]:
df_players_salarios

,Name,Weekly Salary,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,LABEL
0,Lionel Messi,1538905.0,Argentina,RW,RW,170.0,72.0,Left,29.0,Medium / Medium,...,68.0,71.0,85.0,95.0,88.0,89.0,90.0,74.0,85.0,1.0
1,Cristiano Ronaldo,1248536.0,Portugal,LS,LW,185.0,80.0,Right,32.0,High / Low,...,95.0,85.0,92.0,93.0,90.0,81.0,76.0,85.0,88.0,1.0
2,Antoine Griezmann,997032.0,France,CAM,RS,176.0,67.0,Left,26.0,High / Medium,...,87.0,80.0,81.0,88.0,82.0,84.0,75.0,71.0,87.0,1.0
3,Neymar,797726.0,Brazil,LW,LW,174.0,68.0,Right,25.0,High / Medium,...,61.0,62.0,78.0,89.0,77.0,79.0,84.0,81.0,83.0,1.0
4,Gareth Bale,607706.0,Wales,RS,RW,183.0,74.0,Left,27.0,High / Medium,...,85.0,86.0,91.0,87.0,90.0,86.0,85.0,76.0,76.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1985,Igor Lewczuk,10809.0,Poland,Sub,RCB,187.0,85.0,Right,31.0,High / Medium,...,70.0,62.0,63.0,42.0,35.0,59.0,34.0,52.0,34.0,1.0
2003,Keagan Dolly,10396.0,South Africa,LAM,Sub,170.0,70.0,Left,24.0,High / Medium,...,65.0,23.0,58.0,68.0,73.0,70.0,69.0,54.0,61.0,1.0
2037,Vid Belec,10070.0,Slovenia,Sub,Sub,192.0,84.0,Right,26.0,Medium / Medium,...,75.0,13.0,23.0,10.0,16.0,11.0,13.0,23.0,10.0,1.0
2090,Stefanos Kapino,9613.0,Greece,Sub,GK,196.0,91.0,Right,23.0,Medium / Medium,...,71.0,12.0,20.0,13.0,19.0,14.0,18.0,26.0,19.0,1.0


In [162]:
df_players_salarios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 339 entries, 0 to 2197
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               339 non-null    object 
 1   Weekly Salary      339 non-null    float64
 2   Nationality        339 non-null    object 
 3   National_Position  339 non-null    object 
 4   Club_Position      339 non-null    object 
 5   Height             339 non-null    float64
 6   Weight             339 non-null    float64
 7   Preffered_Foot     339 non-null    object 
 8   Age                339 non-null    float64
 9   Work_Rate          339 non-null    object 
 10  Weak_foot          339 non-null    float64
 11  Skill_Moves        339 non-null    float64
 12  Ball_Control       339 non-null    float64
 13  Dribbling          339 non-null    float64
 14  Marking            339 non-null    float64
 15  Sliding_Tackle     339 non-null    float64
 16  Standing_Tackle    339 no

In [164]:
from sklearn.linear_model import LinearRegression

In [412]:
### Código aquí ###
cols = df_players_salarios.columns
cols_categorical = []
cols_num = [c for c in cols if (c not in cols_categorical) and (c not in ["Name", "Weekly Salary", "Nationality", "Preffered_Foot", "Work_Rate", "National_Position", "Club_Position"])]

ct = ColumnTransformer([("Standard", MinMaxScaler(), cols_num), ("OneHotEncoder", OneHotEncoder(handle_unknown='ignore'), cols_categorical)])

In [413]:
from sklearn.feature_selection import SelectKBest, chi2, VarianceThreshold

In [445]:
linear_regression = LinearRegression()
pipe_linear_regression = Pipeline([
    ('scaler', ct),
    ('regression', linear_regression)
    ])

In [446]:
### Código aquí ###
X, y = df_players_salarios[cols_categorical+cols_num], df_players_salarios["Weekly Salary"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=69)

In [447]:
%%time
pipe_linear_regression.fit(X_train, y_train)

Wall time: 7.99 ms


Pipeline(steps=[('scaler',
                 ColumnTransformer(transformers=[('Standard', MinMaxScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Weak_foot', 'Skill_Moves',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                

In [448]:
pipe_linear_regression.score(X_train, y_train)

0.4529240254830784

In [449]:
pipe_linear_regression.score(X_test, y_test)

0.3624028559932517

In [450]:
%%time
linear_regression_pred = pipe_linear_regression.predict(X_test)

Wall time: 4.96 ms


### Obtener un regresor con un $𝑅^2$ superior a 0.35.

In [451]:
from sklearn.metrics import r2_score

In [452]:
r2_score(y_test, linear_regression_pred)

0.3624028559932517

In [453]:
px.scatter(x=y_test, y=linear_regression_pred)

### Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>